# Back-Testing for Algorithmic Trading Strategies

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yahoofinancials as yf
from yahoo_fin.stock_info import *
import requests_html
import requests
import ftplib
import ta


<b>Let's get the historical time series data of the stock by specifying the start and end dates<b>

In [8]:
history = yf.YahooFinancials('ASELS.IS').get_historical_price_data('2021-01-01', '2021-12-31', 'daily')
df = pd.DataFrame(history['ASELS.IS']['prices'])
df.head()

,date,high,low,open,close,volume,adjclose,formatted_date
0,1609745400,18.610001,18.299999,18.350000,18.360001,36331500,18.098007,2021-01-04
1,1609831800,18.510000,18.180000,18.320000,18.280001,33769846,18.019148,2021-01-05
2,1609918200,18.950001,18.430000,18.500000,18.709999,73963507,18.443010,2021-01-06
3,1610004600,19.170000,18.790001,18.799999,18.950001,66513392,18.679588,2021-01-07
4,1610091000,19.150000,18.900000,19.059999,18.980000,32197137,18.709156,2021-01-08


In [9]:
df.drop('date', axis=1, inplace=True)

In [10]:
df.index = pd.to_datetime(df['formatted_date'])
df.drop('formatted_date', axis=1, inplace=True)